In [16]:
import pickle

In [30]:
class Name:
    def __set__(self, instance, value):
        if not isinstance(value, str) or not value.isalpha():
            print('Вы ввели что-то не то')
            raise AttributeError
        instance._name = value
    def __get__(self, instance, owner):
        return instance._name

class Phone:
    def __set__(self, instance, value):
        if not isinstance(value, str) or not value.isdigit():
            print('Вы ввели что-то не то')
            raise AttributeError
        instance._phone = value
    def __get__(self, instance, owner):
        return instance._phone
    
    


In [31]:
class PetName:
    def __set__(self, instance, value):
        if not isinstance(value, str) or not value.isalpha():
            print('Вы ввели что-то не то')
            raise AttributeError
        instance._name = value
    def __get__(self, instance, owner):
        return instance._name
class Age:
    def __set__(self, instance, value):
        if not isinstance(value, int) or value < 0 or value > 20: # предположим, что животное не живет больще 20 лет
            print('Вы ввели что-то не то')
            raise AttributeError
        instance._age = value
    def __get__(self, instance, owner):
        return instance._age
    
    

In [32]:
class Client:
    name = Name()
    phone = Phone()
    def __init__(self):
        self.__name = None
        self.__phone = None
        self.pets = []


    def add_pet(self, pet):
        self.pets.append(pet)

        
    def __repr__(self):
        return f'Client(name: {self.name}, phone: {self.phone})'
    

class Pet:
    name = PetName()
    age = Age()
    def __init__(self):
        self.__name = None
        self.__age = None
        self.appointments = [] # записи на прошедшие приемы
        self.current_appointment = None # наличие записи на текущий прием
    
   


    def set_current_appointment(self, date): # записаться на прием
        self.current_appointment = date

    def done(self, comment): # завершение приема
        if self.current_appointment:
            self.appointments.append((self.current_appointment, comment)) # в прошедшие приемы сохраняется этот прием и комментарий к нему
            self.current_appointment = None # текущего приема больше нет

    def __repr__(self):
       return f'Pet(name: {self.name}, age: {self.age}, current_appointment: {self.current_appointment}, appointments: {self.appointments})'
    


In [33]:
class Database:
    def __init__(self):
        self.clients = []
    
    def add_client(self, client):
        self.clients.append(client)

    def find_client(self, name=None, phone=None): # искать клиента по имени или номеру телефона
        for client in self.clients:
          if (name and client.name == name) or (phone and client.phone == phone):
            return client
        return None
    
    def show_appointments(self):
        for client in self.clients:
            for pet in client.pets:
                if pet.appointments: # чтобы попадали только питомцы, уже побывавшие на приеме
                    print(pet)

    def save(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

            


In [34]:
class Programm:
    def __init__(self):
        self.database = Database()

    def start(self):
        while True:
           choice = input('Выберите действие:\n1.Добавит клиента\n2.Найти лиента в базе\n3.Посмотреть записи о прошедших приемах\n4.Проверить запись\n5. Записаться на прием\n6.Отметить, что прием завершился\n7.Выйти') 
           if choice == '1':
                client = Client()
                name = input('Введите имя клиента')
                client.name = name
                phone = input('Введите телефон клиента')
                client.phone = phone
                while True: # чтобы одному хозяину можно было добавить несколько питомцев
                    pet_name = input('Введите имя питомца')
                    if pet_name == '':
                        break
                    pet_age = int(input('Введите возраст питомца'))
                    pet = Pet()
                    pet.name = pet_name
                    pet.age = pet_age
                    client.add_pet(pet) 
                self.database.add_client(client)
           elif choice == '2':
                name_phone = input('Введите имя или телефон клиента')
                client = self.database.find_client(name=name_phone) or self.database.find_client(phone=name_phone)
                if client:
                    print(client)
                    print('Питомцы клиента')
                    for pet in client.pets:
                        print(pet)
                else:
                    print('Клиент не найден')
           elif choice == '3':
                self.database.show_appointments()
           elif choice == '4':
                name_phone = input('Введите имя или телефон клиента')
                client = self.database.find_client(name=name_phone) or self.database.find_client(phone=name_phone)
                if client:
                    pet_name = input('Введите имя питомца')
                    pet = next((pet for pet in client.pets if pet.name == pet_name), None)
                    if pet:
                        if pet.current_appointment:
                            print(f'У питомца {pet.name} есть запись на прием: {pet.current_appointment}')
                        else:
                            print(f'У питомца нет текущей записи')
                    else:
                        print('Питомец не найден')
                else:
                    print('Клиент не найден')
           elif choice == '5':
                name_phone = input('Введите имя или телефон клиента')
                client = self.database.find_client(name=name_phone) or self.database.find_client(phone=name_phone)
                if client:
                    pet_name = input('Введите имя питомца')
                    pet = next((pet for pet in client.pets if pet.name == pet_name), None)
                    if pet:
                        appointment_date = input('Введите дату приема')
                        pet.set_current_appointment(appointment_date)
                    else:
                        print('Питомец не найден')
                else:
                    print('Клиент не найден')
           elif choice == '6':
                name_phone = input('Введите имя или телефон клиента')
                client = self.database.find_client(name=name_phone) or self.database.find_client(phone=name_phone)
                if client:
                    pet_name = input('Введите имя питомца')
                    pet = next((pet for pet in client.pets if pet.name == pet_name), None)
                    if pet:
                        comment = input('Введите комментарий по приему')
                        pet.done(comment)
                    else:
                        print('Питомец не найден')
                else:
                    print('Клиент не найден')
           elif choice == '7':
                filename = 'clinic'
                self.database.save(filename)
                break
            
            

In [35]:
programm = Programm()
programm.start()

Client(name: Женя, phone: 12345)
Питомцы клиента
Pet(name: Жорж, age: 3, current_appointment: None, appointments: [])
Client(name: Женя, phone: 12345)
Питомцы клиента
Pet(name: Жорж, age: 3, current_appointment: None, appointments: [])
Pet(name: Жорж, age: 3, current_appointment: None, appointments: [('25.12.2024', 'норм')])
У питомца нет текущей записи
Client(name: Женя, phone: 12345)
Питомцы клиента
Pet(name: Жорж, age: 3, current_appointment: None, appointments: [('25.12.2024', 'норм')])
